## Freedmen's Bureau Documents Analysis

In [2]:
import os
from openai import OpenAI
import ollama
import pandas as pd

In [2]:
df = pd.read_csv('archive/contract-records.csv')
df.head()

,project_id,category,sub_category,transcription_text,document_url
0,11406,Contracts,Apprenticeship Agreement,Copy_x000D_\n_x000D_\nAgreement of Apprentices...,https://transcription.si.edu/transcribe/11406/...
1,11406,Contracts,Apprenticeship Agreement,[H 132 ENCL] _x000D_\n_x000D_\nCopy_x000D_\n_x...,https://transcription.si.edu/transcribe/11406/...
2,11406,Contracts,Apprenticeship Agreement,is agreed on the part of the party of the firs...,https://transcription.si.edu/transcribe/11406/...
3,11406,Contracts,Apprenticeship Agreement,continuance of this agreement & finally that t...,https://transcription.si.edu/transcribe/11406/...
4,15369,Contracts,Apprenticeship Agreement,"[[preprinted]]_x000D_\nBureau of Refugees, Fre...",https://transcription.si.edu/transcribe/15369/...


### LLM configuration for python (OpenAI API, Ollama)

- OpenAI

In [5]:
client = OpenAI(
    api_key = os.environ["OPENAI_API_KEY"],
    )
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Why is the sky blue?",
        }
    ],
    model="gpt-3.5-turbo",
)

- Ollama

In [3]:
response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue to us because of a phenomenon called scattering, which involves the interaction between sunlight and the tiny molecules of gases in the atmosphere.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When the sun shines, it sends out a wide range of electromagnetic radiation, including visible light, ultraviolet (UV) rays, and infrared (IR) radiation.
2. **Scattering occurs**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. **Short wavelengths scattered more**: When light hits these gas molecules, some of it scatters in all directions. This scattering effect is more pronounced for shorter (blue) wavelengths than longer (red) wavelengths.
4. **Blue light reaches our eyes**: As a result of this scattering, the blue light from the sun becomes dispersed and reaches our eyes from all parts of the sky. The blue color domina

### Parsing sentences

In [19]:
inputs = [
    "John James agrees to pay $50/month to RJ Hampshire for work on the Farm",
    "Elizabeth James will pay $30 per month to Levi Rodgers for Gardening",
    "Johnson Ollaman will pay $1.25 per day to both John Smith and Jane Smith for teaching the children of the community",
    "Claire Daniels charges $50 weekly to local community members for cooking classes, emphasizing the joy of healthy eating.",
    "Marcus Wellby commits to donating $500 annually to the Green Earth Foundation for environmental conservation efforts.",
    "Dr. Helena Russell charges $100 per hour for providing guidance and support to medical students, aiming to enhance their clinical skills and knowledge.",
    "The local sports club agrees to pay $75 each to coaches Sarah Miller, Danny Glover, Alex Reed, and Jamie Fox for conducting a weekend sports clinic.",
    """This Agreement made this 14th day of August A.D. 1865, by and between F.R.J. Terry of the county of Copiah and State of Mississippi of the first part, and the person hereinafter named and undersigned,
    Freedmen of the second part [[?]] That for the purpose of working in the [[?]] known as Beagley's [[?]] Yard in the county aforesaid for two months commencing on the 14th day of August 1865 and terminating
    on the 14th day of October 1865. The said F.R.J. Terry party of the first part, in consideration of the [[?]] and conditions hereinafter mentioned on the part of the party of the second part agrees to pay said
    laborer "10" ten dollars per month and furnish free of charge clothing and good of good quality and sufficient quantity, good and sufficient quarters, and kind and humane treatment. And it is further agreed
    that in case the said F.R.J. Terry shall fail, neglect, or refuse to fulfill any of the obligations assumed by him, he shall besides the legal recourse left to the party aggrieved render this contract liable
    to amendment by the Provost Marshal of Freedmen. And it is agreed on the part of the party of second part that he will well and faithfully perform such labor as the said F.R.J. Terry may require of him for the
    time aforesaid, nor exceeding ten hours per day in summer and nine hours in winter. And in case the said laborer shall absent himself from or refuse to perform the labor herein promised, he shall loose the time
    and be punished as such manner as the Provost Marshal shall deem propper.""",
]

outputs = [
    [{"payer": "John James", "recipient": "RJ Hampshire", "amount": 50, "pay frequency": "monthly", "description": "farming"}],
    [{"payer": "Elizabeth James", "recipient": "Levi Rodgers", "amount": 30, "pay frequency": "monthly", "description": "gardening"}],
    [{"payer": "Johnson Ollaman", "recipient": "John Smith", "amount": 1.25, "pay frequency": "daily", "description": "teaching the children of the community"}, {"payer": "Johnson Ollaman", "recipient": "Jane Smith", "amount": 1.25, "pay frequency": "daily", "description": "teaching the children of the community"}],
    [{"payer": "Claire Daniels", "recipient": "Local community members", "amount": 50, "pay frequency": "weekly", "description": "cooking classes"}],
    [{"payer": "Marcus Wellby", "recipient": "Green Earth Foundation", "amount": 500, "pay frequency": "yearly", "description": "donation for environmental conservation"}],
    [{"payer": "Dr. Helena Russell", "recipient": "Medical students", "amount": 100, "pay frequency": "hourly", "description": "mentorship and clinical skill enhancement"}],
    [{"payer": "The local sports club", "recipient": "Sarah Miller", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Danny Glover", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Alex Reed", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Jamie Fox", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"}],
    [{"payer": "F.R.J. Terry", "payee": "Freedmen", "amount":10, "pay frequency": "monthly", "description": "working in the yard"}],
]

In [29]:
system_message = """From any given sentcence, grab a payer, recipient, amout, pay frequuency and description. Output shoud be as JSON format. 
Note that there may be a few recipients (if they are being paid, each should have a full sepearate entry). Do not write anything around, I just need plain information"""

response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'system',
    'content': system_message
  },
  {
    'role': 'user',
    'content': inputs[0]
  },
  {
    'role': 'assistant',
    'content': str(outputs[0])
  },
  {
    'role': 'user',
    'content': "The local sports club agrees to pay $75 each to coaches Sarah Miller, Danny Glover, Alex Reed, and Jamie Fox for conducting a weekend sports clinic."
  }
])
print(response['message']['content'])

[{'payer': 'local sports club', 'recipient': 'Sarah Miller', 'amount': 75, 'pay frequency': 'one-time', 'description': 'sports clinic'}, 
 {'payer': 'local sports club', 'recipient': 'Danny Glover', 'amount': 75, 'pay frequency': 'one-time', 'description': 'sports clinic'}, 
 {'payer': 'local sports club', 'recipient': 'Alex Reed', 'amount': 75, 'pay frequency': 'one-time', 'description': 'sports clinic'}, 
 {'payer': 'local sports club', 'recipient': 'Jamie Fox', 'amount': 75, 'pay frequency': 'one-time', 'description': 'sports clinic'}]
